In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

In [2]:
# Sample dataset: User-item ratings
data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    'item_id': [101, 102, 103, 101, 103, 104, 102, 104, 101, 105, 103, 105],
    'rating': [5, 3, 4, 4, 5, 2, 3, 4, 5, 3, 2, 4]
}
df = pd.DataFrame(data)

In [3]:
# Create user-item matrix
user_item_matrix = df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)


In [4]:
# Convert to numpy array
matrix = user_item_matrix.values
user_mean = np.mean(matrix, axis=1)
normalized_matrix = matrix - user_mean[:, np.newaxis]

In [5]:
# Apply Singular Value Decomposition (SVD)
U, sigma, Vt = svds(normalized_matrix, k=2)
sigma = np.diag(sigma)

In [6]:
# Reconstruct the predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_mean[:, np.newaxis]
predictions_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

In [7]:
# Function to recommend items for a user
def recommend_items(user_id, num_recommendations=3):
    user_predictions = predictions_df.loc[user_id].sort_values(ascending=False)
    already_rated = df[df['user_id'] == user_id]['item_id'].tolist()
    recommendations = user_predictions.drop(already_rated).head(num_recommendations)
    return recommendations

In [8]:
#Recommend items for user 2
print("Recommended items for User 2:")
print(recommend_items(2))

Recommended items for User 2:
item_id
102    1.603745
105   -0.124624
Name: 2, dtype: float64
